This notebook processes the CSV file doorvaarten.csv downloaded from www.soundtoll.nl. <br>
It fetches JSON files containing details about column selection and renaming from the STRO 2.0 GitHub repository. <br>

In [ ]:
import requests
import pandas as pd
import json

In [ ]:
# path to downloaded copy of doorvaarten.csv
doorvaarten = r"C:\STRO10\doorvaarten.csv\doorvaarten.csv" # update filepath if necessary

# 1. Preparation

In [ ]:
# retrieve files for preparing entity creation from the STRO 2.0 GitHub repository
owner = 'dhofu'
repo = 'stro20'
column_selection = 'STRO_20_column_selection'
rename_columns = 'STRO_20_column_names'
url_selection = f"https://api.github.com/repos/{owner}/{repo}/contents/{column_selection}"
url_rename = f"https://api.github.com/repos/{owner}/{repo}/contents/{rename_columns}"

In [ ]:
response_column_selection = requests.get(url_selection)
response_rename_columns = requests.get(url_rename)

In [ ]:
# fetch the files for selecting columns from the 'doorvaarten' dataframe
selection_mappings = {
    'usecols_customs_entry': None,
    'usecols_master': None,
    'usecols_remarks': None,
    'usecols_ship': None,
    'usecols_taxes_master': None,
    'usecols_taxes_totals': None,
}
for file in response_column_selection.json():
    if file['name'].startswith('doorvaarten_'):
        key = file['name'].split('_', 1)[1].split('.', 1)[0]
        if key in selection_mappings:
            selection_mappings[key] = requests.get(file['download_url']).json()

usecols_customs_entry = selection_mappings['usecols_customs_entry']
usecols_master = selection_mappings['usecols_master']
usecols_remarks = selection_mappings['usecols_remarks']
usecols_ship = selection_mappings['usecols_ship']
usecols_taxes_master = selection_mappings['usecols_taxes_master']
usecols_taxes_totals = selection_mappings['usecols_taxes_totals']

In [ ]:
# fetch the files for renaming columns from the 'doorvaarten' dataframe 
rename_mappings = {
    'rename_customs_entry': None,
    'rename_master': None,
    'rename_remarks_entries': None,
    'rename_ship': None,
    'rename_taxes_master': None,
    'rename_taxes_totals': None
}

for file in response_rename_columns.json():
    if file['name'].startswith('doorvaarten_'):
        key = file['name'].split('_', 1)[1].split('.', 1)[0]
        if key in rename_mappings:
            rename_mappings[key] = requests.get(file['download_url']).json()

rename_customs_entry = rename_mappings['rename_customs_entry']
rename_master = rename_mappings['rename_master']
rename_remarks_entries = rename_mappings['rename_remarks_entries']
rename_ship = rename_mappings['rename_ship']
rename_taxes_master = rename_mappings['rename_taxes_master']
rename_taxes_totals = rename_mappings['rename_taxes_totals']

# 2. Create Entities

## 2.1. Create dataframes

In [ ]:
# create dataframe customs entry
# this is the core unit of information in the database
df_customs_entry = pd.read_csv(doorvaarten, sep=",", quotechar='"', usecols=usecols_customs_entry, encoding="utf-8", low_memory=False).\
rename(columns=rename_customs_entry).reset_index(drop=True)

In [ ]:
# create the new entity 'master'
# this entity contains name data about the master as well as information about his domicile
# the information about the domicile is 'raw'; the data are entered as found in the source
# low_memory=False is added to suppress dtype warning
df_master = pd.read_csv(doorvaarten, sep=",", quotechar='"', usecols=usecols_master, encoding="utf-8", low_memory=False).\
rename(columns=rename_master).reset_index(drop=True)

In [ ]:
# create new entity 'remarks about the customs entry'
# this entity contains three types of remarks: remarks made by the keyer; remarks about the source; and help 
# low_memory=False is added to suppress dtype warning
df_remarks_entry = pd.read_csv(doorvaarten, sep=",", quotechar='"', usecols=usecols_remarks, encoding="utf-8", low_memory=False).\
rename(columns=rename_remarks_entries).reset_index(drop=True)

In [ ]:
# create new entity 'ship'
# this entity contains information about the ship 
# low_memory=False is added to suppress dtype warning
df_ship = pd.read_csv(doorvaarten, sep=",", quotechar='"', usecols=usecols_ship, encoding="utf-8", low_memory=False).\
rename(columns=rename_ship).reset_index(drop=True)

In [ ]:
# create new entity 'taxes payable by the master'
# this entity contains information about the taxes payable by the master
# low_memory=False is added to suppress dtype warning
df_taxes_master = pd.read_csv(doorvaarten, sep=",", quotechar='"', usecols=usecols_taxes_master, encoding="utf-8", low_memory=False).\
rename(columns=rename_taxes_master).reset_index(drop=True)

In [ ]:
# create new entity 'taxes totals'
# this entity contains information about the calculation steps for taxation
# low_memory=False is added to suppress dtype warning
df_taxes_totals = pd.read_csv(doorvaarten, sep=",", quotechar='"', usecols=usecols_taxes_totals, encoding="utf-8", low_memory=False).\
rename(columns=rename_taxes_totals).reset_index(drop=True)

## 2.2. Refactor dataframes

In [ ]:
melted_df_remarks_entry = df_remarks_entry.melt(id_vars=['ce_id'], value_vars=['remarkSource', 'remarkKeyer', 'help'])

In [ ]:
melted_df_remarks_entry.rename(columns={'variable': 'remarkType'}, inplace=True)

In [ ]:
melted_df_remarks_entry.dropna(inplace=True)

In [ ]:
melted_df_remarks_entry

In [ ]:
df_ship.dropna(inplace=True)

## 2.3. Process tax data

In [ ]:
# in df_taxes_master currency3 and quantity3 are empty and can be removed
df_taxes_master.drop(columns=['Currency3', 'Quantity3'], inplace=True)

In [ ]:
melted_df_taxes_master_cry = df_taxes_master.melt(id_vars=['ce_id', 'discountAnnotation'], value_vars=['Currency1', 'Currency2'])

In [ ]:
melted_df_taxes_master_cry

In [ ]:
melted_df_taxes_master_qty = df_taxes_master.melt(id_vars=['ce_id', 'discountAnnotation'], value_vars=['Quantity1', 'Quantity2'])

In [ ]:
melted_df_taxes_master_qty

In [ ]:
melted_df_taxes_master = melted_df_taxes_master_cry.merge(melted_df_taxes_master_qty, how='left', left_index=True, right_index=True)

In [ ]:
melted_df_taxes_master.drop(columns=['ce_id_y', 'discountAnnotation_y', 'variable_x', 'variable_y'], inplace=True)

In [ ]:
melted_df_taxes_master.rename(columns={'ce_id_x': 'ce_id', 'discountAnnotation_x': 'discountAnnotation', 'value_x': 'currency', 'value_y': 'quantity'}, inplace=True)

In [ ]:
melted_df_taxes_master.dropna(inplace=True)

In [ ]:
melted_df_taxes_master

In [ ]:
# column-wise review of the dataframe with count() and value_counts()
# many NaN values, but no columns that can be removed entirely
# same procedure as above (df_taxes_master and melt) - all currency-columns are processed first; then all quantities columns
df_taxes_totals_cry = df_taxes_totals.melt(id_vars=['ce_id'], value_vars=['subtotal1Currency1', 'subtotal1Currency2', 'subtotal1Currency3', \
                                                                         'subtotal2Currency1', 'subtotal2Currency2', 'subtotal2Currency3', \
                                                                         'totalCurrency1', 'totalCurrency2', 'totalCurrency3', \
                                                                         'totalCurrency4', 'totalCurrency5'])

In [ ]:
df_taxes_totals_qty = df_taxes_totals.melt(id_vars=['ce_id'], value_vars=['subtotal1Quantity1', 'subtotal1Quantity2', 'subtotal1Quantity3', \
                                                                         'subtotal2Quantity1', 'subtotal2Quantity2', 'subtotal2Quantity3', \
                                                                         'totalQuantity1', 'totalQuantity2', 'totalQuantity3', \
                                                                         'totalQuantity4', 'totalQuantity5'])

In [ ]:
melted_df_taxes_totals = df_taxes_totals_cry.merge(df_taxes_totals_qty, how='left', left_index=True, right_index=True)

In [ ]:
melted_df_taxes_totals

In [ ]:
melted_df_taxes_totals.drop(columns=['ce_id_y', 'variable_y'], inplace=True)

In [ ]:
melted_df_taxes_totals.rename(columns={'ce_id_x': 'ce_id', 'variable_x': 'calculationStep', 'value_x': 'currency', 'value_y': 'quantity'}, inplace=True)

In [ ]:
melted_df_taxes_totals.dropna(inplace=True)

In [ ]:
# check the melted dataframe
# one final step could be to clean up the values in 'calculationStep' by renaming them subtotal1Currency1 => subtotal1 etc.
# simple syntax: df.loc[ df[“column_name”] == “some_value”, “column_name”] = “value”
melted_df_taxes_totals.loc[melted_df_taxes_totals['calculationStep'] == 'subtotal1Currency1', 'calculationStep'] = 'subtotal1'
melted_df_taxes_totals.loc[melted_df_taxes_totals['calculationStep'] == 'subtotal1Currency2', 'calculationStep'] = 'subtotal1'
melted_df_taxes_totals.loc[melted_df_taxes_totals['calculationStep'] == 'subtotal1Currency3', 'calculationStep'] = 'subtotal1'
melted_df_taxes_totals.loc[melted_df_taxes_totals['calculationStep'] == 'subtotal2Currency1', 'calculationStep'] = 'subtotal2'
melted_df_taxes_totals.loc[melted_df_taxes_totals['calculationStep'] == 'subtotal2Currency2', 'calculationStep'] = 'subtotal2'
melted_df_taxes_totals.loc[melted_df_taxes_totals['calculationStep'] == 'subtotal2Currency3', 'calculationStep'] = 'subtotal2'
melted_df_taxes_totals.loc[melted_df_taxes_totals['calculationStep'] == 'totalCurrency1', 'calculationStep'] = 'total'
melted_df_taxes_totals.loc[melted_df_taxes_totals['calculationStep'] == 'totalCurrency2', 'calculationStep'] = 'total'
melted_df_taxes_totals.loc[melted_df_taxes_totals['calculationStep'] == 'totalCurrency3', 'calculationStep'] = 'total'
melted_df_taxes_totals.loc[melted_df_taxes_totals['calculationStep'] == 'totalCurrency4', 'calculationStep'] = 'total'
melted_df_taxes_totals.loc[melted_df_taxes_totals['calculationStep'] == 'totalCurrency5', 'calculationStep'] = 'total'

In [ ]:
melted_df_taxes_totals.loc[melted_df_taxes_totals['ce_id'] == 281]

In [ ]:
melted_df_taxes_master.loc[melted_df_taxes_master['ce_id'] == 281]

# 3. Save dataframes to disk

In [ ]:
df_customs_entry.to_csv(r"C:\STRO20\customs_entries.csv", sep=';', quotechar= '"', index_label='index', encoding='utf-8')

In [ ]:
df_master.to_csv(r"C:\STRO20\masters.csv", sep=';', quotechar= '"', index_label='index', encoding='utf-8')

In [ ]:
melted_df_remarks_entry.to_csv(r"C:\STRO20\remarks_entries.csv", sep=';', quotechar= '"', index_label='index', encoding='utf-8')

In [ ]:
df_ship.to_csv(r"C:\STRO20\ships.csv", sep=';', quotechar= '"', index_label='index', encoding='utf-8')

In [ ]:
melted_df_taxes_master.to_csv(r"C:\STRO20\taxes_master.csv", sep=';', quotechar= '"', index_label='index', encoding='utf-8')

In [ ]:
melted_df_taxes_totals.to_csv(r"C:\STRO20\taxes_totals.csv", sep=';', quotechar= '"', index_label='index', encoding='utf-8')